 # Лабораторная работа: Обнаружение вредоносных URL-адресов с минимальной размерностью



 ## Описание задачи



 Цель данной работы — построить модель машинного обучения, которая эффективно

 классифицирует URL-адреса на вредоносные и доброкачественные.



 **Особенность:** Модель оценивается не только по точности (F1-score), но и по

 эффективности — чем выше точность при меньшем количестве признаков, тем лучше.



 ## Метрика оценивания



 Ваша оценка учитывает как качество модели (F1-score), так и эффективность

 использования признаков:



 ```

 quality_score = (F1_student - F1_baseline) / (1 - F1_baseline)

 efficiency_bonus = log(N_baseline / N_student) / log(N_baseline)

 final_score = quality_score * (1 + efficiency_bonus)

 ```



 где:

 - `F1_student` — F1-score вашей модели (macro)

 - `F1_baseline` — F1-score базовой модели (минимально допустимое значение)

 - `N_student` — количество признаков в вашей модели

 - `N_baseline` — количество признаков в baseline модели

 - Если `F1_student < F1_baseline`, то `final_score = 0`



 **Интерпретация:**

 - `quality_score` — оценка качества (0 до 1)

 - `efficiency_bonus` — бонус за сокращение размерности (0 до 1)

 - `final_score` — итоговая оценка с учетом обоих факторов (0 до 2)



 ## Набор данных



 - **Источник:** [URL Dataset](http://www.sysnet.ucsd.edu/projects/url/url_svmlight.tar.gz)

 - **Размер:** ~2.4 млн URL, ~3.2 млн признаков

 - **Формат:** SVM-light (метка +1 = вредоносный, -1 = доброкачественный)

 - **Статья:** [Beyond Blacklists](https://cseweb.ucsd.edu/~jtma/papers/beyondbl-kdd2009.pdf)



 ## Полезные ссылки



 - [Feature Selection (sklearn)](https://scikit-learn.org/stable/modules/feature_selection.html)

 - [Dimensionality Reduction (sklearn)](https://scikit-learn.org/stable/modules/decomposition.html)

 - [Linear Discriminant Analysis (sklearn)](https://scikit-learn.org/stable/modules/lda_qda.html)

 ## Шаг 1: Подготовка данных

In [ ]:
import os
import tarfile
from random import random
from tqdm.auto import tqdm
import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, f1_score

# Распаковка архива с данными
if not os.path.exists("./url_svmlight"):
    print("Распаковка архива...")
    uri = "./url_svmlight.tar.gz"
    tar = tarfile.open(uri, "r:gz")
    tar.extractall()
    tar.close()
    print("Архив распакован!")


 ### Функция разделения на train/test



 Стратифицированное разделение по дням с сохранением баланса классов.

In [ ]:
def train_test_split_data(train_size=0.8):
    """
    Разделяет данные на обучающую и тестовую выборки.
    
    Parameters:
    -----------
    train_size : float
        Доля данных для обучающей выборки (по умолчанию 0.8)
    """
    train = open('./url_svmlight/train.svm', 'w')
    test = open('./url_svmlight/test.svm', 'w')
    summ = 0
    
    for i in tqdm(range(121), desc="Обработка файлов"):
        inn = open('./url_svmlight/Day' + str(i) + '.svm','r')
        
        q = [0, 0]
        
        for line in inn:
            a = line.split()
            if(a[0] == "-1"):
                q[0] += 1
            else:
                q[1] += 1
        
        inn.close()
        summ += q[0] + q[1]
        check = [int(train_size * q[0]),int(train_size * q[1])]
        start = [0, 0]
        
        inn = open('./url_svmlight/Day' + str(i) + '.svm','r')
        
        if (train_size * q[0]) % 1 >=0.5:
            check[0] += 1
        if (train_size * q[1]) % 1 >=0.5:
            check[1] += 1
        for line in inn:
            a = line.split()

            rand = random()
            if rand > 0.5:
                if a[0] == "-1":
                    if start[0] + 1 <= check[0]:
                        train.write(line)
                        train.write("\n")
                        start[0] += 1
                    else:
                        test.write(line)
                        test.write("\n")
                        q[0] -= 1
                else:
                    if start[1] + 1 <= check[1]:
                        train.write(line)
                        train.write("\n")
                        start[1] += 1
                    else:
                        test.write(line)
                        test.write("\n")
                        q[1] -= 1
            else:
                if(a[0] == "-1"):
                    if q[0] > check[0]:
                        test.write(line)
                        test.write("\n")
                        q[0] -= 1
                    else :
                        train.write(line)
                        train.write("\n")
                        start[0] += 1
                else:
                    if q[1] > check[1]:
                        test.write(line)
                        test.write("\n")
                        q[1] -= 1
                    else :
                        train.write(line)
                        train.write("\n")
                        start[1] += 1
    
    train.close()
    test.close()

# Создание файлов train.svm и test.svm
if not os.path.exists('./url_svmlight/train.svm'):
    print("\nСоздание обучающей и тестовой выборок...")
    train_test_split_data(train_size=0.8)
else:
    print("Файлы train.svm и test.svm уже существуют")


 ## Шаг 2: Загрузка данных и обучение Baseline модели

In [ ]:
n_features = 3231961  # Количество признаков в датасете

print("Загрузка обучающих данных...")
X_train_full, y_train_full = load_svmlight_file( # pyright: ignore[reportAssignmentType]
    "./url_svmlight/train.svm", 
    n_features=n_features
)

print("Загрузка тестовых данных...")
X_test_full, y_test_full = load_svmlight_file( # pyright: ignore[reportAssignmentType]
    "./url_svmlight/test.svm", 
    n_features=n_features
)

print(f"\nРазмер обучающей выборки: {X_train_full.shape}")
print(f"Размер тестовой выборки: {X_test_full.shape}")


In [ ]:
# Обучение baseline модели (LogisticRegression на всех признаках)
print("\nОбучение baseline модели...")
baseline_model = SGDClassifier(loss="log_loss", random_state=42)
baseline_model.fit(X_train_full, y_train_full)

# Предсказания и метрики
y_pred_baseline = baseline_model.predict(X_test_full)
baseline_f1 = f1_score(y_test_full, y_pred_baseline, average="macro")
baseline_features = X_train_full.shape[1]

print("\n" + "="*70)
print("BASELINE MODEL RESULTS")
print("="*70)
print(f"Количество признаков: {baseline_features:,}")
print(f"F1-Score (macro): {baseline_f1:.6f}")
print("\nClassification Report:")
print(classification_report(y_test_full, y_pred_baseline, digits=4))
print("="*70)


 ## Шаг 3: ВАША МОДЕЛЬ — Сокращение размерности и обучение



 **Задание:** Реализуйте свой метод сокращения размерности и обучите модель.



 ### Возможные подходы:



 1. **Feature Selection:**

    - `SelectKBest` — отбор k лучших признаков по статистическому критерию

    - `SelectFromModel` — отбор на основе важности признаков из модели (L1/L2)

    - `RFE` — рекурсивное удаление признаков



 2. **Dimensionality Reduction:**

    - `TruncatedSVD` — сингулярное разложение (работает с разреженными матрицами)

    - `PCA` — метод главных компонент

    - `LDA` — линейный дискриминантный анализ



 3. **Комбинированные подходы:**

    - Сначала Feature Selection, затем Dimensionality Reduction

    - Ансамбли моделей с разными наборами признаков



 ### Шаблон для заполнения:

In [ ]:
# ===== ВАШ КОД НАЧИНАЕТСЯ ЗДЕСЬ =====

# Вы вольны использовать любые методы предобработки данных и добавлять дополнительные этапы

# 1. Сокращение размерности
reducer = ...  # TODO: инициализируйте метод сокращения размерности

# Примените трансформацию к данным
X_train_reduced = ...  # TODO: примените selector к X_train_full
X_test_reduced = ...   # TODO: примените selector к X_test_full

# 2. Обучение модели на сокращенном наборе признаков
# Выберите алгоритм классификации
clf = ...  # TODO: инициализируйте классификатор

# Обучите модель
# TODO: обучите clf на X_train_reduced, y_train_full

# 3. Получение предсказаний
y_pred_reduced = ...  # TODO: получите предсказания для X_test_reduced

# ===== ВАШ КОД ЗАКАНЧИВАЕТСЯ ЗДЕСЬ =====

 ## Шаг 4: Оценка результатов

In [ ]:
# Расчет метрик
f1_reduced = f1_score(y_test_full, y_pred_reduced, average="macro")
n_features_reduced = X_train_reduced.shape[1]

# Расчет финальной оценки с учетом количества признаков
if f1_reduced < baseline_f1:
    quality_score = 0.0
    efficiency_bonus = 0.0
    final_score = 0.0
    status = "❌ НИЖЕ BASELINE"
else:
    # Оценка качества (F1-score)
    quality_score = (f1_reduced - baseline_f1) / (1 - baseline_f1)
    
    # Бонус за эффективность (сокращение размерности)
    if n_features_reduced < baseline_features:
        efficiency_bonus = np.log(baseline_features / n_features_reduced) / np.log(baseline_features)
    else:
        efficiency_bonus = 0.0
    
    # Итоговая оценка
    final_score = quality_score * (1 + efficiency_bonus)
    status = "✓ ВЫШЕ BASELINE"

print("\n" + "="*70)
print("YOUR MODEL RESULTS")
print("="*70)
print(f"Количество признаков: {n_features_reduced:,}")
print(f"Сокращение размерности: {(1 - n_features_reduced/baseline_features)*100:.2f}%")
print(f"F1-Score (macro): {f1_reduced:.6f}")
print("\nClassification Report:")
print(classification_report(y_test_full, y_pred_reduced, digits=4))
print("="*70)

print("\n" + "="*70)
print("FINAL EVALUATION")
print("="*70)
print(f"Baseline F1-Score:  {baseline_f1:.6f} ({baseline_features:,} признаков)")
print(f"Your F1-Score:      {f1_reduced:.6f} ({n_features_reduced:,} признаков)")
print(f"\nУлучшение F1:       {(f1_reduced - baseline_f1):.6f} ({'+' if f1_reduced >= baseline_f1 else ''}{(f1_reduced/baseline_f1 - 1)*100:.2f}%)")
print(f"Сокращение признаков: {baseline_features - n_features_reduced:,} ({(1 - n_features_reduced/baseline_features)*100:.2f}%)")

if f1_reduced >= baseline_f1:
    print(f"\n{'─'*70}")
    print("КОМПОНЕНТЫ ОЦЕНКИ:")
    print(f"  • Quality Score (F1):           {quality_score:.4f}")
    print(f"  • Efficiency Bonus (признаки):  {efficiency_bonus:.4f}")
    print(f"  • Множитель:                    {1 + efficiency_bonus:.4f}x")

print(f"\n{'='*70}")
print(f"ИТОГОВАЯ ОЦЕНКА: {final_score:.4f} {status}")
print(f"{'='*70}")


 ## Рекомендации для улучшения результата



 1. **Экспериментируйте с разными методами** отбора признаков

 2. **Подбирайте гиперпараметры** методов сокращения размерности

 3. **Пробуйте разные классификаторы** (LogisticRegression, SGD, RandomForest и др.)

 4. **Комбинируйте методы** (например, сначала SelectKBest, затем TruncatedSVD)

 5. **Анализируйте важность признаков** для понимания данных



 Удачи в выполнении лабораторной работы! 🚀